In [2]:
import pandas as pd 
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt
from backtester import Backtest_Environment

In [3]:
class BullHighBreakout(Backtest_Environment):
    
    # first line of arguments contains strategy specific parameters, second line contains backtest environment parameters
    def __init__(self, data, mltv, sttv, start_date, end_date,
                 start_cash, pos_size, tc, cooldown=1, 
                 max_executions=None, max_execution_period=None, cash_buffer=0, stoploss=None):
        """
        Constructor method for the strategy child class. Extra arguments specific to the child can be added to this 
        constructor.
        
        The benefit of this is that, strategy specific parameters can be added to this class with the data preparation 
        methods occurring within the class prior to backtest(). 
        
        This structure allows for an easier defintion of new strategies for strategies with the similar rule structures
        but different parameters applied to those rules. 
        """
        
        # Calling the constructor of the parent class - stores input variables + adds strategy agnostic columns to the data
        super().__init__(data, start_cash, pos_size, tc, cooldown, max_executions, max_execution_period, cash_buffer, stoploss)

        # Strategy specific parameters
        self.mltv = mltv
        self.sttv = sttv
        self.start_date = start_date
        self.end_date = end_date
        self.format_strategy_data()
        
    def format_strategy_data(self):
        """
        Prepares data according to the 3 indicators that the strategy requires. The data required is:

        - medium-long term indicator (MLTI): rule 1 - the strategy only initiates a trade when above this value (calculated
        using mltv)
        - short term trade indicator (STTI): rule 2 - the strategy enters positions when the price falls below this value 
        and rule 1 is satisfied (calculated using sttv).
        - position target (PosTarget): rule 3 - the exit conditon relies on this indicator. The strategy exits positions when the 
        price surpasses this value (calculated using sttv). 

        The inputs are:
        - data: any price dataframe containing the columns "Close" and "Open" -
        - mltv: explained above
        - sttv: explained above
        - start_date: self explanatory
        - end_date: self explanatory
        """

        # formatting the name of the close price data (column name utilised in backtest() method of this class)
        self.data = self.data.rename(columns={"Close": "Price"}) # getting current day close in right format for class input

        # STRATEGY SPECIFIC COLUMNS
        self.data["MLTI"] = self.data["Price"].rolling(self.mltv).mean() # medium-long term indicator: 200 day moving average
        self.data["STTI"] = self.data["Price"].rolling(self.sttv).mean() # short term trade indicator
        self.data["PosTarget"] = self.data["Price"].rolling(self.sttv).max() # position target: 1 week high

        # accessing dates of interest
        self.data = self.data.loc[self.start_date:self.end_date] # indexing dataframe between the dates of interest once all indicators are calculated
    
    def process(self, data):
        """
        Takes input data which includes: 
        - a Medium to Long Term Indicator (MLTI)
        - a Short Term Trend Indicator (STTI)
        
        in a pandas Dataframe.
        
        This function runs through the logic behind initialising a trade for this strategy. 
        The variable MTLI is a 200 day MA. The variable STTI is a 7 day MA. 
        
        The Logic for opening positions:
        
        - If the price is greater than the MLTI, we proceed
        - If the price is below the 7 day MA (trending low), proceed 
        - If cash after position opens is greater than buffer: 
            - return num_units: indicating output of the decision process is positive, open a position with number
            of units equal to num_units in backtest()
        """
        
        # long term condition satisfied e.g > 200 day MA
        if data.Price.values[0] > data["MLTI"].values[0]:
            
            # short term condition satisfied: < e.g < 7 day MA
            if data.Price.values[0] < data["STTI"].values[0]:
                
                # calculate number of units
                num_units = int(np.floor((self.pos_size * self.cash_available) / data.Price.values[0]))
                
                # cash check - acts as a lid on opening new positions
                if (self.cash_available - num_units*data.Price.values[0]) > self.cash_buffer:
                    
                    return num_units
                
                else:
                    return 0
            
        return 0

    
    def monitorPosition(self, data, position):
        """
        Takes input of price and position to monitor. Checks if current price is greater than the target price of 
        the position. 
        
        Requires input of:
            - position to monitor
            - current price (in the "data" variable - data variable contains row of that day's
            new data)
            
        The output of running this function is: 
            - True if the position has hit it's target and needs to be closed
            - False if the position has not hit it's target and must remain open
        """
        
                
        # checking if current price is greater than the position's target
        if data.Price.values[0] >= position.PosTarget.values[0]:
                    
            # return true
            return True
        
        # checking if price is less than stoploss
        elif data.Price.values[0] <= position.StopLoss.values[0]:
            
            # return true
            return True
        
        else:
            return False

In [4]:
# strategy specific parameters
crypto = yf.Ticker("BNB-USD").history(period="30y")
mltv = 200
sttv = 5
start_date = pd.to_datetime("2016-01-01")
end_date = pd.to_datetime("2021-10-06")

In [6]:
def rsi(series, lookback):
    """
    This function returns the RSI of an input pandas series.
    The inputs are:
    - series: the input pandas series to calculate the RSI on
    - lookback: the number of days to look back over to calculate the RSI; usually lookback=14
    """
    
    # difference the data and calculate the returns
    delta = series.diff().dropna()

    # obtain the positive and negative returns while clipping out of range values at 0
    up, down = abs(delta.clip(lower=0).rename("Up")), abs(delta.clip(upper=0).rename("Down"))

    # calculate the rolling average gains and losses excluding the 0 values
    avg_gain = up.rolling(lookback).apply(lambda x: x[x!= 0].mean()).rename("Avg Gain").fillna(0)
    avg_loss = down.rolling(lookback).apply(lambda x: x[x!= 0].mean()).rename("Avg Loss").fillna(0)

    # calculating the denominator of the RSI
    RSI_step = 1 + ((avg_gain.shift(1)*(lookback-1)) + up)/((avg_loss.shift(1)*(lookback-1)) + down)

    # obtain the RSI
    RSI = 100 - 100/RSI_step
    
    return RSI 

In [7]:
crypto["RSI"] = rsi(crypto["Close"], 2)
crypto["Prev High"] = crypto["High"].shift(1)
crypto = crypto.dropna()

In [8]:
crypto.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI,Prev High
Date,,,,,,,,,
2017-07-27,0.105108,0.108479,0.100888,0.107737,344499,0,0,100.000000,0.109013
2017-07-28,0.107632,0.109019,0.101473,0.104067,342568,0,0,37.123254,0.108479
2017-07-29,0.104782,0.111264,0.101108,0.107811,340218,0,0,63.347650,0.109019
2017-07-30,0.107935,0.108138,0.103162,0.106414,224261,0,0,42.492339,0.111264
2017-07-31,0.106828,0.108349,0.101600,0.104250,240309,0,0,51.252575,0.108138


In [ ]:
def process(data):
    
    if data.RSI < 15:    
        # calculate number of units
        num_units = int(np.floor((self.pos_size * self.cash_available) / data.Price.values[0]))

        # cash check - acts as a lid on opening new positions
        if (self.cash_available - num_units*data.Price.values[0]) > self.cash_buffer:

            return num_units

        else:
            return 0
            
        return 0
    

In [4]:
strat = BullHighBreakout(aapl, mltv, sttv, start_date, end_date,
                         start_cash=10000, pos_size=0.1, tc=2, cooldown=0, 
                         max_executions=None, max_execution_period=None, cash_buffer=0, stoploss = 0.1)

strat.backtest()

True